In [1]:
import json
with open("static/DATA.json", mode="r", encoding="utf-8") as f:
    DATA = json.load(f)
ldg_names = [e["name"] for e in DATA["DVC_TTHC_LamDong"]["data"]]
lsn_names = [e["name"] for e in DATA["DVC_TTHC_LangSon"]["data"]]

In [4]:
# from pkg.HYSE.HYSE import HYSE_EngineHybrid